In [45]:
import os
from datetime import datetime
from typing import Any, Dict, List

from lib.pg import PgConnect
from pydantic import BaseModel


from datetime import datetime

from lib.pg import PgConnect

from dotenv import load_dotenv
load_dotenv()


pg_warehouse_host = str(os.getenv('PG_WAREHOUSE_HOST') or "")
pg_warehouse_port = int(str(os.getenv('PG_WAREHOUSE_PORT') or 0))
pg_warehouse_dbname = str(os.getenv('PG_WAREHOUSE_DBNAME') or "")
pg_warehouse_user = str(os.getenv('PG_WAREHOUSE_USER') or "")
pg_warehouse_password = str(os.getenv('PG_WAREHOUSE_PASSWORD') or "")



def pg_warehouse_db():
    return PgConnect(
        pg_warehouse_host,
        pg_warehouse_port,
        pg_warehouse_dbname,
        pg_warehouse_user,
        pg_warehouse_password
    )


class CdmRepository:
    def __init__(self, db: PgConnect) -> None:
        self._db = db

    def update_mart(mart_name: str,
                   columns: list,
                   values: list,
                   ) -> None:
        with pg_warehouse_db().connection() as conn:
            values_ = []
            for v in values:
                v_ = f"'{str(v)}'"
                values_.append(v_)
            with conn.cursor() as cur:
                query = f"""
                        INSERT INTO cdm.{mart_name}
                        ({','.join(columns)})
                        VALUES({','.join(values_)})
                        ON CONFLICT (user_id, category_id)
                        DO UPDATE 
                        SET order_cnt= cdm.{mart_name}.order_cnt + EXCLUDED.order_cnt;

                    """
                print(query)
                cur.execute(query

                            )
                conn.commit()

In [46]:
print(pg_warehouse_host
,pg_warehouse_port
,pg_warehouse_dbname
,pg_warehouse_user
,pg_warehouse_password)

rc1b-v9fvv9q03peguu4n.mdb.yandexcloud.net 6432 sprint9dwh sprint9dwh sprint9dwh


In [50]:
import json

# Открываем файл и считываем данные
with open('/Users/andrewkomkov/PycharmProjects/yandex_de/de-project-sprint-9/dds_test/messages-2.json', 'r') as f:
    data = json.load(f)
data

{'mart': 'user_category_counters',
 'user_id': 'df72256c-0c8b-5438-8be7-8d59a0acee4d',
 'category_id': '0f6d48bd-fe1f-51f6-a39f-31fe8f5e52a5',
 'category_name': 'Выпечка',
 'order_cnt': 4}

In [51]:
def update_cdm(msg):
    mart_name = msg['mart']
    msg.pop('mart')
    columns = list(msg.keys())
    values = list(msg.values())
    print(columns)
    print(values)
    CdmRepository.update_mart(mart_name,columns,values)
update_cdm(data)

['user_id', 'category_id', 'category_name', 'order_cnt']
['df72256c-0c8b-5438-8be7-8d59a0acee4d', '0f6d48bd-fe1f-51f6-a39f-31fe8f5e52a5', 'Выпечка', 4]

                        INSERT INTO cdm.user_category_counters
                        (user_id,category_id,category_name,order_cnt)
                        VALUES('df72256c-0c8b-5438-8be7-8d59a0acee4d','0f6d48bd-fe1f-51f6-a39f-31fe8f5e52a5','Выпечка','4')
                        ON CONFLICT (user_id, category_id)
                        DO UPDATE 
                        SET order_cnt= cdm.user_category_counters.order_cnt + EXCLUDED.order_cnt;

                    


In [10]:
# Хаб с категориями
def put_category_hub(msg):
    categories = []
    payload = msg['payload']
    for product in payload['products']:
        categories.append(product['category'])
    for category in categories:
        columns = ['category_name', 'load_src']
        values = [category,'stg-service-orders']
        DdsRepository.insert_hub('category',columns,values)

# Хаб с заказами
def put_order_hub(msg):
    if msg['object_type'] == 'order':
        payload = msg['payload']

        order_id = payload['id']
        order_dt = payload['date']
        columns = ['order_id','order_dt','load_src']

        values = [order_id,order_dt,'stg-service-orders']

        DdsRepository.insert_hub('order',columns,values)
    else:
        pass

# Хаб с продуктами
def put_product_hub(msg):
    payload = msg['payload']
    products = []
    for product in payload['products']:
        products.append(product['id'])
    for product_id in products:
        columns = ['product_id', 'load_src']
        values = [product_id,'stg-service-orders']
        DdsRepository.insert_hub('product',columns,values)

# Хаб с ресторанами
def put_restaurant_hub(msg):
    restaurant_id = msg['payload']['restaurant']['id']
    columns = ['restaurant_id', 'load_src']
    values = [restaurant_id,'stg-service-orders']
    DdsRepository.insert_hub('restaurant',columns,values)

# Хаб с пользователями
def put_user_hub(msg):
    user_id = msg['payload']['user']['id']
    columns = ['user_id', 'load_src']
    values = [user_id,'stg-service-orders']
    DdsRepository.insert_hub('user',columns,values)


In [11]:
# Линк order_product
def put_order_product_link(msg):
    payload = msg['payload']
    order_id = payload['id']
    products = []
    for product in payload['products']:
        products.append(product['id'])
    for product_id in products:
        DdsRepository.insert_link('order',
                    order_id,
                    'product',
                    product_id,
                   'stg-service-orders',
                   )

# Линк order_user
def put_order_user_link(msg):
    payload = msg['payload']
    order_id = payload['id']
    user_id = payload['user']['id']
    DdsRepository.insert_link('order',
                    order_id,
                    'user',
                    user_id,
                   'stg-service-orders',
                   )
        
# Линк product_category
def put_product_category_link(msg):
    payload = msg['payload']
    for product in payload['products']:
        DdsRepository.insert_link('product',
                    product['id'],
                    'category',
                    product['category'],
                   'stg-service-orders',
                   )
        
# Линк product_restaurant
def put_product_restaurant_link(msg):
    payload = msg['payload']
    restaurant_id = payload['restaurant']['id']
    products = []
    for product in payload['products']:
        products.append(product['id'])
    for product_id in products:
        DdsRepository.insert_link('product',
                    product_id,
                    'restaurant',
                    restaurant_id,
                   'stg-service-orders',
                   )

In [14]:
# Сателлит order_cost
def put_order_cost_satellit(msg):
    hub_name = 'order'
    satellit_name = 'cost'
    payload = msg['payload']
    order_id = payload['id']
    order_cost = payload['cost']
    order_payment = payload['payment']
    columns = [f'h_{hub_name}_pk','cost','payment','load_src']
    values = [order_id,order_cost,order_payment,'stg-']
    DdsRepository.insert_satellit(hub_name,
                    satellit_name,
                    columns,
                    values,
                    )
    
# Сателлит order_status
def put_order_status_satellit(msg):
    hub_name = 'order'
    satellit_name = 'status'
    payload = msg['payload']
    order_id = payload['id']
    order_status = payload['status']
    columns = [f'h_{hub_name}_pk','status','load_src']
    values = [order_id,order_status,'stg-']
    DdsRepository.insert_satellit(hub_name,
                    satellit_name,
                    columns,
                    values,
                    )
    
# Сателлит product_names
def put_product_name_satellit(msg):
    hub_name = 'product'
    satellit_name = 'names'
    payload = msg['payload']
    for product in payload['products']:
        product_id = product['id']
        product_name = product['name']
        columns = [f'h_{hub_name}_pk','name','load_src']
        values = [product_id,product_name,'stg-']
    DdsRepository.insert_satellit(hub_name,
                    satellit_name,
                    columns,
                    values,
                    )
    
# Сателлит restaurant_names
def put_restaurant_name_satellit(msg):
    hub_name = 'restaurant'
    satellit_name = 'names'
    payload = msg['payload']
    restaurant_id = payload['restaurant']['id']
    restaurant_name = payload['restaurant']['name']
    columns = [f'h_{hub_name}_pk','name','load_src']
    values = [restaurant_id,restaurant_name,'stg-']
    DdsRepository.insert_satellit(hub_name,
                    satellit_name,
                    columns,
                    values,
                    )
    
# Сателлит user_names
def put_user_names_satellit(msg):
    hub_name = 'user'
    satellit_name = 'names'
    payload = msg['payload']
    user_id = payload['user']['id']
    username = payload['user']['name']
    userlogin = payload['user']['name']
    columns = [f'h_{hub_name}_pk','username','userlogin','load_src']
    values = [user_id,username,userlogin,'stg-']
    DdsRepository.insert_satellit(hub_name,
                    satellit_name,
                    columns,
                    values,
                    )

In [15]:
put_order_cost_satellit(data)
put_order_status_satellit(data)
put_product_name_satellit(data)
put_restaurant_name_satellit(data)
put_user_names_satellit(data)


                        INSERT INTO dds.s_order_cost
                        (hk_order_cost_pk,h_order_pk,cost,payment,load_src)
                        VALUES('8f4e0448-125a-56cf-b0b1-10db8f2751fe','6154c506-6f77-58dd-a845-913103ff5d22','1560','1560','stg-')
                        ON CONFLICT (h_order_pk)
                        DO UPDATE 
                        SET hk_order_cost_pk = EXCLUDED."hk_order_cost_pk",h_order_pk = EXCLUDED."h_order_pk",cost = EXCLUDED."cost",payment = EXCLUDED."payment",load_src = EXCLUDED."load_src",load_dt = CURRENT_TIMESTAMP;

                    

                        INSERT INTO dds.s_order_status
                        (hk_order_status_pk,h_order_pk,status,load_src)
                        VALUES('420d1f73-5897-5b99-b611-41782e66c56a','6154c506-6f77-58dd-a845-913103ff5d22','CLOSED','stg-')
                        ON CONFLICT (h_order_pk)
                        DO UPDATE 
                        SET hk_order_status_pk = EXCLUDED."hk_order_statu

In [ ]:
put_order_product_link(data)
put_order_user_link(data)
put_product_category_link(data)
put_product_restaurant_link(data)

In [ ]:
put_category_hub(data)
put_order_hub(data)
put_product_hub(data)
put_restaurant_hub(data)
put_user_hub(data)

In [ ]:
name = 'category'
names = ['category_name', 'load_dt', 'load_src']
values = ['test','2020-02-04','jp']
DdsRepository.insert_hub(name,names,values)

In [ ]:
name = 'order'
names = ['order_id', 'order_dt', 'load_dt', 'load_src']
values = ['3','2020-03-04','2020-03-04','jjpp']
DdsRepository.insert_hub(name,names,values)